In [ ]:
%run environmental_variables.ipynb

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from casadi import *
from casadi.tools import *

import warnings

mpl.rcParams['font.size'] = 16

In [ ]:
# Function to generate temperature and radiation trajectories for a specified number of days
def generate_weather_trajectory(t, dt):
    
    t = t*24  # convert days to hours
    
    time_trajectory = np.linspace(0, t, int(t/dt))
    
    T_summer = []
    qrad_summer = []
    T_winter = []
    qrad_winter = []
    
    for time in time_trajectory:
        T_summer.append(T_env_summer(time))
        qrad_summer.append(q_rad_summer(time))
        T_winter.append(T_env_winter(time))
        qrad_winter.append(q_rad_winter(time))
    
    return T_summer, qrad_summer, T_winter, qrad_winter, time_trajectory

In [ ]:
warnings.filterwarnings('ignore')

t_sim = 7 # simulation time in days days

dt = 1/4 # sampling time in hours

h = dt*3600 # discretization step in seconds

ts, qs, tw, qw, tt = generate_weather_trajectory(t_sim, dt)

N_sim = len(tt) # set simulation time for later

In [ ]:
# Visualize Parameters
fig, ax = plt.subplots(1,4, figsize=(30, 5))

ax[0].plot(tt, np.array(ts)-273.15)
ax[1].plot(tt, qs)
ax[2].plot(tt, np.array(tw)-273.15)
ax[3].plot(tt, qw)

x_lim = [0,t_sim*24]
for m in range(4): ax[m].set_xlim(*x_lim)
ax[1].set_ylim([-0.1,1.5]); ax[3].set_ylim([-0.1,1.5])
    
for m in range(4): ax[m].grid();

plt.show()

In [ ]:
# System Constants
V_R = 75                # m^3
V_w_sep = 4             # m^3
V_w_inf = 4             # m^3
qdot_H = 75e-3          # kW
alpha_R_W = 7.7e-3      # kW/(m^2.K)
F_ij = 5/3600           # m^3/s
A_w_sep = 12            # m^2
A_w_inf = 15            # m^2
qdot_PC = 120e-3        # kW
alpha_W_inf = 2.3e-3    # kW/(m^2.K)
zeta_R = 1.225          # kg.m^-3
zeta_W = 1700           # kg.m^-3
c_p_R = 1               # kJ.kg^-3.K^-1
c_p_W = 1               # kJ.kg^-3.K^-1
beta = 0.0875
N_H = [2, 1, 3]
N_PC = [1, 2, 2]

In [ ]:
# Initialize states, inputs and parameters in Casadi
n_rooms = 3
n_walls = 10

# States:
T_R = SX.sym("T_R", n_rooms)
T_W = SX.sym("T_W", n_walls)
# Structure of T_W
# T_W = ([ T_W_room0_e, T_W_room1_e, T_W_room0_n, T_W_room0_s, T_W_room0_w, 
#                                    T_W_room1_n, T_W_room1_s, T_W_room2_n, T_W_room2_e, T_W_room2_s])
# Note that:
#     T_W_room0_e = T_W_room1_w
#     T_W_room1_e = T_W_room2_w 

# Inputs: 
Qdot_heat = SX.sym("Qdot_heat", n_rooms)
Qdot_cool = SX.sym("Qdot_cool", n_rooms)

# Parameters
T_env = SX.sym("T_env")
qdot_rad = SX.sym("qdot_rad")

In [ ]:
# System ODEs

# Helpers to formulate differential equations of room temperatures
sum_D = [
    T_R[1]-T_R[0],
    (T_R[0]-T_R[1])+(T_R[2]-T_R[1]),
    T_R[1]-T_R[2]
]
sum_AT = [
    A_w_inf*(T_W[2]-T_R[0])+A_w_sep*(T_W[0]-T_R[0])+A_w_inf*(T_W[3]-T_R[0])+A_w_inf*(T_W[4]-T_R[0]),
    A_w_inf*(T_W[5]-T_R[1])+A_w_sep*(T_W[1]-T_R[1])+A_w_inf*(T_W[6]-T_R[1])+A_w_sep*(T_W[0]-T_R[1]),
    A_w_inf*(T_W[7]-T_R[2])+A_w_inf*(T_W[8]-T_R[2])+A_w_inf*(T_W[9]-T_R[2])+A_w_sep*(T_W[1]-T_R[2])
]

# Differential equations for room temperatures
T_R_dot = []
for i in range(3):
    T_R_dot.append((F_ij/V_R)*sum_D[i] + (alpha_R_W/(zeta_R*c_p_R*V_R))*sum_AT[i] 
                   + (N_H[i]*qdot_H+N_PC[i]*qdot_PC+Qdot_heat[i]-Qdot_cool[i])/(zeta_R*c_p_R*V_R))

# Differential equations for inner wall temperatures, only T_W_room0_e and T_W_room1_e are considered 
# (since T_W_room0_e = T_W_room1_w and T_W_room1_e = T_W_room2_w)
T_W_dot_inner = [
    (1/zeta_W*c_p_W)*(A_w_sep/V_w_sep)*alpha_R_W*((T_R[0]-T_W[0])+(T_R[1]-T_W[0])),  # T_W_room0_e
    (1/zeta_W*c_p_W)*(A_w_sep/V_w_sep)*alpha_R_W*((T_R[1]-T_W[1])+(T_R[2]-T_W[1])),  # T_W_room1_e
]

# Differential equations for outer wall temperatures
T_W_dot_outer = [
    (1/zeta_W*c_p_W)*(A_w_inf/V_w_inf)*(alpha_R_W*(T_R[0]-T_W[2])+alpha_W_inf*(T_env-T_W[2])+beta*qdot_rad),
    (1/zeta_W*c_p_W)*(A_w_inf/V_w_inf)*(alpha_R_W*(T_R[0]-T_W[3])+alpha_W_inf*(T_env-T_W[3])+beta*qdot_rad),
    (1/zeta_W*c_p_W)*(A_w_inf/V_w_inf)*(alpha_R_W*(T_R[0]-T_W[4])+alpha_W_inf*(T_env-T_W[4])+beta*qdot_rad),
    (1/zeta_W*c_p_W)*(A_w_inf/V_w_inf)*(alpha_R_W*(T_R[1]-T_W[5])+alpha_W_inf*(T_env-T_W[5])+beta*qdot_rad),
    (1/zeta_W*c_p_W)*(A_w_inf/V_w_inf)*(alpha_R_W*(T_R[1]-T_W[6])+alpha_W_inf*(T_env-T_W[6])+beta*qdot_rad),
    (1/zeta_W*c_p_W)*(A_w_inf/V_w_inf)*(alpha_R_W*(T_R[2]-T_W[7])+alpha_W_inf*(T_env-T_W[7])+beta*qdot_rad),
    (1/zeta_W*c_p_W)*(A_w_inf/V_w_inf)*(alpha_R_W*(T_R[2]-T_W[8])+alpha_W_inf*(T_env-T_W[8])+beta*qdot_rad),
    (1/zeta_W*c_p_W)*(A_w_inf/V_w_inf)*(alpha_R_W*(T_R[2]-T_W[9])+alpha_W_inf*(T_env-T_W[9])+beta*qdot_rad),
]

In [ ]:
x_sim = vertcat(T_R, T_W)

u_sim = vertcat(Qdot_heat, Qdot_cool)

p_sim = vertcat(T_env, qdot_rad)

T_states_dot = vertcat(*T_R_dot, *T_W_dot_inner, *T_W_dot_outer)

system = Function("sys", [x_sim, u_sim, p_sim], [T_states_dot])

In [ ]:
ode = {'x': x_sim, 'ode': T_states_dot, 'p': vertcat(u_sim, p_sim)}

opts = {'tf': h}

ode_solver = integrator('F', 'idas', ode, opts)

In [ ]:
# Method for plotting the results of the simulation
def plot_states(res_x, Tenv, q_rad, u_devices=None, plot_constraints=False):

    res_x_plt = res_x.T-273.15
    Tenv = np.array(Tenv)-273.15
    q_rad = q_rad

    time_tr = np.arange(0,t_sim*24+dt,dt)

    fig, axs = plt.subplots(6,7, figsize=(16,11))

    for ax in axs.flat:
        ax.tick_params(axis='both', which='major', labelsize=11.5)  # Major ticks
        ax.set_xticks(np.linspace(0,t_sim*24,3))
        ax.set_xlim([0,t_sim*24+dt])  

    for ax in axs[:3,:].flat:
        ax.set_ylabel('T ($^\circ$C)', fontsize=9.5)
        ax.set_xlabel('Time (hours)', fontsize=9.5)

    for ax in axs[3:5,:].flat:
        ax.set_ylabel('Power (kW)', fontsize=9.5)
        ax.set_xlabel('Time (hours)', fontsize=9.5)

    axs[0,1].plot(time_tr, res_x_plt[:,5], label='T_W0n'); axs[0,1].legend(loc='best', prop={'size': 9.5})
    axs[0,3].plot(time_tr, res_x_plt[:,8], label='T_W1n'); axs[0,3].legend(loc='best', prop={'size': 9.5})
    axs[0,5].plot(time_tr, res_x_plt[:,10], label='T_W2n'); axs[0,5].legend(loc='best', prop={'size': 9.5})

    axs[1,0].plot(time_tr, res_x_plt[:,7], label='T_W0w'); axs[1,0].legend(loc='best', prop={'size': 9.5})
    axs[1,1].plot(time_tr, res_x_plt[:,0], label='T_R0'); axs[1,1].legend(loc='best', prop={'size': 9.5})
    axs[1,2].plot(time_tr, res_x_plt[:,3], label='T_W0e,\nT_W1w'); axs[1,2].legend(loc='best', prop={'size': 9.5})
    axs[1,3].plot(time_tr, res_x_plt[:,1], label='T_R1'); axs[1,3].legend(loc='best', prop={'size': 9.5})
    axs[1,4].plot(time_tr, res_x_plt[:,4], label='T_W1e,\nT_W2w'); axs[1,4].legend(loc='best', prop={'size': 9.5})
    axs[1,5].plot(time_tr, res_x_plt[:,2], label='T_R2'); axs[1,5].legend(loc='best', prop={'size': 9.5})
    axs[1,6].plot(time_tr, res_x_plt[:,11], label='T_W2e'); axs[1,6].legend(loc='best', prop={'size': 9.5})

    axs[2,1].plot(time_tr, res_x_plt[:,6], label='T_W0s'); axs[2,1].legend(loc='best', prop={'size': 9.5})
    axs[2,3].plot(time_tr, res_x_plt[:,9], label='T_W1s'); axs[2,3].legend(loc='best', prop={'size': 9.5})
    axs[2,5].plot(time_tr, res_x_plt[:,12], label='T_W2s'); axs[2,5].legend(loc='best', prop={'size': 9.5})

    if u_devices is not None:
        axs[3,1].plot(time_tr, u_devices.T[:,0], label='u_heat0'); axs[3,1].legend(loc='best', prop={'size': 9.5})
        axs[3,3].plot(time_tr, u_devices.T[:,1], label='u_heat1'); axs[3,3].legend(loc='best', prop={'size': 9.5})
        axs[3,5].plot(time_tr, u_devices.T[:,2], label='u_heat2'); axs[3,5].legend(loc='best', prop={'size': 9.5})
        axs[4,1].plot(time_tr, u_devices.T[:,3], label='u_cool0'); axs[4,1].legend(loc='best', prop={'size': 9.5})
        axs[4,3].plot(time_tr, u_devices.T[:,4], label='u_cool1'); axs[4,3].legend(loc='best', prop={'size': 9.5})
        axs[4,5].plot(time_tr, u_devices.T[:,5], label='u_cool2'); axs[4,5].legend(loc='best', prop={'size': 9.5})
    else:
        axs[3,1].plot(time_tr, np.zeros(res_x_plt.shape[0]), label='u_heat0'); axs[3,1].legend(loc=1, prop={'size': 9.5})
        axs[3,3].plot(time_tr, np.zeros(res_x_plt.shape[0]), label='u_heat1'); axs[3,3].legend(loc=1, prop={'size': 9.5})
        axs[3,5].plot(time_tr, np.zeros(res_x_plt.shape[0]), label='u_heat2'); axs[3,5].legend(loc=1, prop={'size': 9.5})
        axs[4,1].plot(time_tr, np.zeros(res_x_plt.shape[0]), label='u_cool0'); axs[4,1].legend(loc=1, prop={'size': 9.5})
        axs[4,3].plot(time_tr, np.zeros(res_x_plt.shape[0]), label='u_cool1'); axs[4,3].legend(loc=1, prop={'size': 9.5})
        axs[4,5].plot(time_tr, np.zeros(res_x_plt.shape[0]), label='u_cool2'); axs[4,5].legend(loc=1, prop={'size': 9.5})

    gs = axs[5, 1].get_gridspec()
    for ax in axs[5, 1:3]:
        ax.remove()
    axbig = fig.add_subplot(gs[5, 1:3])
    axbig.plot(tt, Tenv, label='Environment Temperature'); axbig.set_xlim([0,t_sim*24])
    axbig.tick_params(axis='both', which='major', labelsize=11.5)
    axbig.legend(loc=1, prop={'size': 10})
    axbig.set_ylabel('T ($^\circ$C)', fontsize=9.5)
    axbig.set_xlabel('Time (hours)', fontsize=9.5)

    gs = axs[5, 4].get_gridspec()
    for ax in axs[5, 4:6]:
        ax.remove()
    axbig = fig.add_subplot(gs[5, 4:6])
    axbig.plot(tt, q_rad, label='Solar Radiation'); axbig.set_xlim([0,t_sim*24])
    axbig.tick_params(axis='both', which='major', labelsize=11.5)
    axbig.legend(loc=1, prop={'size': 10})
    axbig.set_ylabel('Radiation ($kW/m^2$)', fontsize=9.5)
    axbig.set_xlabel('Time (hours)', fontsize=9.5)

    axs[0,0].set_visible(False); axs[0,2].set_visible(False); axs[0,4].set_visible(False); axs[0,6].set_visible(False)
    axs[2,0].set_visible(False); axs[2,2].set_visible(False); axs[2,4].set_visible(False); axs[2,6].set_visible(False)
    axs[3,0].set_visible(False); axs[3,2].set_visible(False); axs[3,4].set_visible(False); axs[3,6].set_visible(False)
    axs[4,0].set_visible(False); axs[4,2].set_visible(False); axs[4,4].set_visible(False); axs[4,6].set_visible(False)
    axs[5,0].set_visible(False); axs[5,1].set_visible(False); axs[5,3].set_visible(False); axs[5,5].set_visible(False); axs[5,6].set_visible(False)

    if plot_constraints:
        T_R_ub = 23
        T_R_lb = 19
        T_W_ub = 40
        T_W_lb = 5
        Q_dot_ub = 3

        axs[0,1].axhline(T_W_ub, color='red', linestyle='--'); axs[0,1].axhline(T_W_lb, color='red', linestyle='--')
        axs[0,3].axhline(T_W_ub, color='red', linestyle='--'); axs[0,3].axhline(T_W_lb, color='red', linestyle='--')
        axs[0,5].axhline(T_W_ub, color='red', linestyle='--'); axs[0,5].axhline(T_W_lb, color='red', linestyle='--')

        axs[1,0].axhline(T_W_ub, color='red', linestyle='--'); axs[1,0].axhline(T_W_lb, color='red', linestyle='--')
        axs[1,1].axhline(T_R_ub, color='red', linestyle='--'); axs[1,1].axhline(T_R_lb, color='red', linestyle='--')
        axs[1,2].axhline(T_W_ub, color='red', linestyle='--'); axs[1,2].axhline(T_W_lb, color='red', linestyle='--')
        axs[1,3].axhline(T_R_ub, color='red', linestyle='--'); axs[1,3].axhline(T_R_lb, color='red', linestyle='--')
        axs[1,4].axhline(T_W_ub, color='red', linestyle='--'); axs[1,4].axhline(T_W_lb, color='red', linestyle='--')
        axs[1,5].axhline(T_R_ub, color='red', linestyle='--'); axs[1,5].axhline(T_R_lb, color='red', linestyle='--')
        axs[1,6].axhline(T_W_ub, color='red', linestyle='--'); axs[1,6].axhline(T_W_lb, color='red', linestyle='--')

        axs[2,1].axhline(T_W_ub, color='red', linestyle='--'); axs[2,1].axhline(T_W_lb, color='red', linestyle='--')
        axs[2,3].axhline(T_W_ub, color='red', linestyle='--'); axs[2,3].axhline(T_W_lb, color='red', linestyle='--')
        axs[2,5].axhline(T_W_ub, color='red', linestyle='--'); axs[2,5].axhline(T_W_lb, color='red', linestyle='--')

        axs[3,1].axhline(Q_dot_ub, color='red', linestyle='--')
        axs[3,3].axhline(Q_dot_ub, color='red', linestyle='--')
        axs[3,5].axhline(Q_dot_ub, color='red', linestyle='--')
        axs[4,1].axhline(Q_dot_ub, color='red', linestyle='--')
        axs[4,3].axhline(Q_dot_ub, color='red', linestyle='--')
        axs[4,5].axhline(Q_dot_ub, color='red', linestyle='--')

    plt.tight_layout()
    plt.show()

In [ ]:
# Simulate the exact system autonomously using sundials integrator
def sundials_simulation(x_sim_0, T_env, q_rad):

    N_sim = len(T_env)
    res_x_sundials = [x_sim_0]

    u_devices_sim = np.zeros(6)

    for i in range(N_sim):
        u_sim_0 = vertcat(u_devices_sim, T_env[i], q_rad[i])
        res_integrator = ode_solver(x0=x_sim_0, p=u_sim_0)
        x_sim_next = res_integrator['xf']
        res_x_sundials.append(x_sim_next)
        x_sim_0 = x_sim_next

    # Make an array from the list of arrays:
    res_x_sundials = np.concatenate(res_x_sundials, axis=1)

    return res_x_sundials

In [ ]:
T_R_0 = 20 # initial Temperature for rooms in Celcius
T_W_0 = 18 # initial Temperature for walls in Celcius

# Initial condition for the states 
x_sim_0 = vertcat(np.ones(n_rooms)*(T_R_0+273.15), np.ones(n_walls)*(T_W_0+273.15))

res_x_sundials_summer = sundials_simulation(x_sim_0, ts, qs)
res_x_sundials_winter = sundials_simulation(x_sim_0, tw, qw)

In [ ]:
# fig, ax = plt.subplots(2, figsize=(10,10), sharex=True)
# # ax.plot(np.arange(0,t_sim*24,dt),res_x_sundials.T-273.15)
# ax[0].plot(res_x_sundials_summer.T-273.15)
# ax[0].set_ylabel('Temperature')
# ax[0].grid()
# ax[0].set_title(f"{t_sim} days in the summer")
# ax[1].plot(res_x_sundials_winter.T-273.15)
# ax[1].set_ylabel('Temperature')
# ax[1].set_xlabel('Time')
# ax[1].set_title(f"{t_sim} days in the winter")
# ax[1].grid()
# fig.suptitle("Simulation using Sundials Integrator")
# plt.show()

# fig, ax = plt.subplots(figsize=(10,6))
# plot_labels = ['TR0', 'TR1', 'TR2', 'TW0e', 'TW1e', 'TW0n', 'TW0s', 'TW0w', 'TW1n', 'TW1s', 'TW2n', 'TW2e', 'TW2s']
# for i in range(3):
#     ax.plot(res_x_sundials_summer[i,:]-273.15, label=plot_labels[i])
# ax.legend()
# ax.set_ylabel('temperatures')
# ax.set_xlabel('time in hours')
# ax.grid()

# Plot autonomous behaviour for 7 days in the summer
plot_states(res_x_sundials_summer, Tenv=ts, q_rad=qs)

In [ ]:
# Plot autonomous behaviour for 7 days in the winter
plot_states(res_x_sundials_winter, Tenv=tw, q_rad=qw)

In [ ]:
# Implicit Euler Discretization (without objective function, just to verify that the discretization works)
# g => X_k+1 = x_k + h*f(X_k+1, u_k, d_k)

nx = x_sim.shape[0] # number of states
nu = u_sim.shape[0] # number of inputs
nd = p_sim.shape[0] # number of parameters

# Step size for discretization
# h = dt

# Initialize empty list of constraints
g = []

x_k_next = SX.sym('x_k_next', nx) # next state x_k+1
x_k = SX.sym('x_k', nx) # current state x_k
u_k = SX.sym('u_k', nu) # control input
d_k = SX.sym('d_k', nd) # parameters

# Append implicit euler formula as constraint
gk = x_k_next - x_k - h*system(x_k_next, u_k, d_k)
g.append(gk)

# concatenate constraints to a vector
g = vertcat(*g)

# dictionary for nlpsol object (no objective function required)
nlp = {'x':x_k_next.reshape((-1,1)), 'g':g, 'p':vertcat(x_k, u_k, d_k)}

# create nlpsol object (with ipopt)
S = nlpsol('S', 'ipopt', nlp)


In [ ]:
# Simulate the system autonomously using euler discretization scheme
def euler_simulation(x_sim_0, T_env, q_rad):

    N_sim = len(T_env)
    res_x_euler = [x_sim_0]
    u_devices_sim = np.zeros(6)

    # Simulate the system with Euler-discretized model
    for i in range(N_sim):
        u_sim_0 = vertcat(u_devices_sim,T_env[i],q_rad[i])
        res_euler = S(lbg=0, ubg=0, p=vertcat(x_sim_0, u_sim_0))
        x_sim_next = res_euler['x'].full().reshape(nx,1, order='F')
        res_x_euler.append(x_sim_next)
        x_sim_0 = x_sim_next

    res_x_euler = np.concatenate(res_x_euler,axis=1)

    return res_x_euler

In [ ]:
x_sim_0 = vertcat(np.ones(n_rooms)*(T_R_0+273.15), np.ones(n_walls)*(T_W_0+273.15))

res_x_euler_summer = euler_simulation(x_sim_0, ts, qs)
res_x_euler_winter = euler_simulation(x_sim_0, tw, qw)

In [ ]:
# Plot the results

# fig, ax = plt.subplots(2, figsize=(10,10), sharex=True)
# # ax.plot(np.arange(0,t_sim*24,dt),res_x_sundials.T-273.15)
# ax[0].plot(res_x_euler_summer.T-273.15)
# ax[0].set_ylabel('Temperature')
# ax[0].grid()
# ax[0].set_title(f"{t_sim} days in the summer")
# ax[1].plot(res_x_euler_winter.T-273.15)
# ax[1].set_ylabel('Temperature')
# ax[1].set_xlabel('Time')
# ax[1].set_title(f"{t_sim} days in the winter")
# ax[1].grid()
# fig.suptitle("Simulation with Implicit Euler Discretization")
# plt.show()

# Plot autonomous behaviour for 7 days in the summer
plot_states(res_x_euler_summer, Tenv=ts, q_rad=qs)

In [ ]:
# Plot autonomous behaviour for 7 days in the winter
plot_states(res_x_euler_winter, Tenv=tw, q_rad=qs)

In [ ]:
# plot the error in discretization
fig, ax = plt.subplots(2, figsize=(10,6))
ax[0].plot(res_x_euler_summer.T-res_x_sundials_summer.T)
ax[1].plot(res_x_euler_winter.T-res_x_sundials_winter.T)
fig.suptitle("Error between Sundials and Euler Simulation")
plt.show()
# we can see that the error is small and bounded

In [ ]:
# System Matrices
A = simplify(jacobian(T_states_dot, x_sim))

B = simplify(jacobian(T_states_dot, u_sim))

E = simplify(jacobian(T_states_dot, p_sim))

# F = simplify(T_states_dot - (mtimes(A, x_sim) + mtimes(B, u_sim) + mtimes(E, p_sim)))

A = DM(A).toarray()
B = DM(B).toarray()
E = DM(E).toarray()
# F = DM(F).toarray()

# Discrete System Matrices
A_tilde = np.linalg.inv(np.eye(nx) - h*A)
B_tilde = A_tilde@(h*B)
E_tilde = A_tilde@(h*E)
# F_tilde = A_tilde@(h*F)

In [ ]:
# Grid search for the minimum number of measured states:

def find_rank(C,A_1):
    O = C
    for k in range(1,n_x):
        O = np.concatenate((O, C@np.linalg.matrix_power(A_1,k)))
    Rank = np.linalg.matrix_rank(O)
    return Rank

n_x = 13
C1 = np.zeros((5,n_x))
for i in range(13):
    if(i==0):
        C1[0,12] = 0
    else:
        C1[0,i-1] = 0
    C1[0,i] = 1
    for j in range(13):
        if(j==0):
            C1[1,12]=0
        else:
            C1[1,j-1] =0
        C1[1,j] = 1
        for k in range(13):
            if(k==0):
                C1[2,12] = 0
            else:
                C1[2,k-1] = 0
            C1[2,k] = 1
            for l in range(13):
                if(l==0):
                    C1[3,12] = 0
                else:
                    C1[3,l-1] = 0
                C1[3,l] = 1
                for m in range(13):
                    if(m==0):
                        C1[4,12] = 0
                    else:
                        C1[4,m-1] = 0
                    C1[4,m] = 1
                    fi = find_rank(C1,A)
                    if(fi==13):
                        print(C1)

In [ ]:
# Observability Analysis

def observability_matrix(A,C):
    nx = A.shape[0]
    O = C
    
    for k in range(1,nx):
        O = np.concatenate((O, C@np.linalg.matrix_power(A,k)))
        
    return O

# Measure 5 states as minimum
C1 = np.zeros(13).reshape(1,13)
C1[0,5] = 1
C2 = np.zeros(13).reshape(1,13)
C2[0,6] = 1
C3 = np.zeros(13).reshape(1,13)
C3[0,9] = 1
C4 = np.zeros(13).reshape(1,13)
C4[0,10] = 1
C5 = np.zeros(13).reshape(1,13)
C5[0,11] = 1

C = np.vstack((C1, C2, C3, C4, C5))
ny = C.shape[0]

# Get the observability matrix
O = observability_matrix(A,C)

# Rank of the observability matrix
Rank = np.linalg.matrix_rank(O)
print(f'Rank = {Rank}')

In [ ]:
def kalman_estimator(x0_exact, x0_observer, u0, d0, P0, var_x, var_y):

    # Kalman Matrices
    Q_kalman = var_x * np.eye(nx)
    R_kalman = var_y * np.eye(ny)

    # Gaussian noise added to the process and the measurements
    w_x = np.random.normal(0, np.sqrt(var_x), nx).reshape(nx, 1)
    w_y = np.random.normal(0, np.sqrt(var_y), ny).reshape(ny, 1)

    # Simulated measurement
    y = C @ x0_exact + w_y

    # Prediction
    res_euler = S(lbg=0, ubg=0, p=vertcat(x0_observer, u0, d0))
    x0_observer = res_euler['x'].full().reshape(nx,1, order='F') + w_x
    P0 = A_tilde @ P0 @ A_tilde.T + Q_kalman
    
    # Correction
    Gain_0 = P0 @ C.T @ np.linalg.inv(C @ P0 @ C.T + R_kalman) # 3
    x0_observer = x0_observer + Gain_0 @ (y - C @ x0_observer) # 4
    P0 = (np.eye(nx) - Gain_0 @ C) @ P0 # 5

    return x0_observer, P0, y

In [ ]:
# Kalman Filter Simulation

# Initial condition of real process
x_sim_0 = vertcat(np.ones(n_rooms)*(T_R_0+273.15), np.ones(n_walls)*(T_W_0+273.15))

# Initial condition for observer
x0_observer = vertcat(np.ones(n_rooms)*(25+273.15), np.ones(n_walls)*(23+273.15))

var_x = 1e-5
var_y = 1e-2

Q_kalman = var_x * np.eye(nx)
R_kalman = var_y * np.eye(ny)
P0 = np.eye(nx)

u_devices_sim = np.zeros(6)

# matrices to store the states 
x_data = [x_sim_0]
x_hat_data = [x0_observer]
y_measured = []

for i in range(N_sim):

    # Generate the input
    u_sim_0 = vertcat(u_devices_sim, ts[i],qs[i])

    # Simulate the real system 
    res_integrator = ode_solver(x0=x_sim_0, p=u_sim_0)
    x_0 = res_integrator['xf']
    x_data.append(x_0)
    x_sim_0 = x_0

    d0 = np.array([ts[i]] + [qs[i]])
    x0_observer, P0, y = kalman_estimator(x_0, x0_observer, u_devices_sim, d0, P0, var_x, var_y)
    y_measured.append(y)
    x_hat_data.append(x0_observer)

res_x_exact = np.concatenate(x_data, axis=1)
res_x_hat = np.concatenate(x_hat_data, axis=1)
res_y_measured = np.concatenate(y_measured, axis=1)


In [ ]:
# Plot results of the Kalman Filter:
res_x_exact_plt = res_x_exact.T - 273.15
res_x_hat_plt = res_x_hat.T - 273.15
res_y_measured_plt = res_y_measured.T - 273.15

fig, ax = plt.subplots(figsize=(10,6), sharex=True)
ax.plot(res_x_exact_plt[:,5])
ax.plot(res_x_hat_plt[:,5], '--')
# ax.plot(res_y_measured_plt[:,0], 'c.')
plt.show()

fig, axs = plt.subplots(3,7, figsize=(16,6))

for ax in axs.flat:
    ax.tick_params(axis='both', which='major', labelsize=8)  # Major ticks
    ax.tick_params(axis='both', which='minor', labelsize=6)  # Minor ticks

axs[0,1].plot(res_x_exact_plt[:,5]); axs[0,1].plot(res_x_hat_plt[:,5], '--'); axs[0,1].plot(res_y_measured_plt[:,0], 'c.')
axs[0,3].plot(res_x_exact_plt[:,8]); axs[0,3].plot(res_x_hat_plt[:,8], '--')
axs[0,5].plot(res_x_exact_plt[:,10]); axs[0,5].plot(res_x_hat_plt[:,10], '--'); axs[0,5].plot(res_y_measured_plt[:,3], 'c.')

axs[1,0].plot(res_x_exact_plt[:,7]); axs[1,0].plot(res_x_hat_plt[:,7], '--'); 
axs[1,1].plot(res_x_exact_plt[:,0]); axs[1,1].plot(res_x_hat_plt[:,0], '--')
axs[1,2].plot(res_x_exact_plt[:,3]); axs[1,2].plot(res_x_hat_plt[:,3], '--')
axs[1,3].plot(res_x_exact_plt[:,1]); axs[1,3].plot(res_x_hat_plt[:,1], '--')
axs[1,4].plot(res_x_exact_plt[:,4]); axs[1,4].plot(res_x_hat_plt[:,4], '--')
axs[1,5].plot(res_x_exact_plt[:,2]); axs[1,5].plot(res_x_hat_plt[:,2], '--')
axs[1,6].plot(res_x_exact_plt[:,11]); axs[1,6].plot(res_x_hat_plt[:,11], '--'); axs[1,6].plot(res_y_measured_plt[:,4], 'c.')

axs[2,1].plot(res_x_exact_plt[:,6]); axs[2,1].plot(res_x_hat_plt[:,6], '--'); axs[2,1].plot(res_y_measured_plt[:,1], 'c.')
axs[2,3].plot(res_x_exact_plt[:,9]); axs[2,3].plot(res_x_hat_plt[:,9], '--'); axs[2,3].plot(res_y_measured_plt[:,2], 'c.')
axs[2,5].plot(res_x_exact_plt[:,12]); axs[2,5].plot(res_x_hat_plt[:,12], '--')

axs[0,0].set_visible(False); axs[0,2].set_visible(False); axs[0,4].set_visible(False); axs[0,6].set_visible(False)
axs[2,0].set_visible(False); axs[2,2].set_visible(False); axs[2,4].set_visible(False); axs[2,6].set_visible(False)

plt.tight_layout()
plt.show()

# plot_states(res_x_hat, ts, qs)

In [ ]:
N_hours = 10 # Prediction Horizon in hours
N = int(N_hours/dt) # Prediction Horizon in steps

# Weighting matrices:
Q = 20
Q = np.diag([Q]*n_rooms + [0]*n_walls)

R = 10
R = np.diag(R*np.ones(nu))

c = 10
c = DM(np.ones(nu)*c)

T_R_ref = 20  # degrees C; reference room temperature for the MPC to track

x_ref = np.array(n_rooms*[T_R_ref+273.15] + n_walls*[0])

u_k_prev = SX.sym('u_k_prev', nu)

# Pseudo-Economic Stage Cost function:
pseudo_stage_cost = (x_k - x_ref).T@Q@(x_k - x_ref) + (u_k - u_k_prev).T@R@(u_k - u_k_prev) + c.T@u_k
pseudo_stage_cost_fcn = Function('pseudo_stage_cost_fcn', [x_k, u_k, u_k_prev], [pseudo_stage_cost])

# Economic Stage Cost Function:
economic_stage_cost = (u_k - u_k_prev).T@R@(u_k - u_k_prev) + c.T@u_k
economic_stage_cost_fcn = Function('economic_stage_cost_fcn', [u_k, u_k_prev], [economic_stage_cost])

# Terminal Cost function:
# terminal_cost = (x_k - x_ref).T@Q@(x_k - x_ref)
# terminal_cost_fcn = Function('terminal_cost_fcn', [x_k], [terminal_cost])

In [ ]:
# MPC with implicit euler discretization and pseudo-economic/economic cost function:
# Optimization problem formuation:

# Optimization variables symbolic structure
opt_x = struct_symSX([
    entry('x', shape=nx, repeat=[N+1]),
    entry('u', shape=nu, repeat=[N]),
    entry('eps', shape=n_rooms, repeat=[2]) # slack variables for room temperatures for soft constraints
])

# Penalty Parameter for slack terms of room temperature bounds for soft constraints:
slack_penalty = 1e2

# State and Input Constraints:
T_R_lb_soft = 19+273.15 # lower bound of room temperature
T_R_ub_soft = 23+273.15 # upper bound of room temperature
T_R_lb = -np.inf
T_R_ub = np.inf
T_W_lb = 5 # lower bound of wall temperature
T_W_ub = 40 # upper bound of wall temperature
Qdot_lb = 0 # minimum heating and cooling duty
Qdot_heat_ub = 3 # maximum heating duty (KW)
Qdot_cool_ub = 3 # maximum cooling duty (KW)
eps_lb = -np.inf
eps_ub = np.inf

# Symbolic Casadi structures for state and input constraints
lb_opt_x = opt_x(0)
ub_opt_x = opt_x(0)

# Set the bounds for states
for i in range(N+1):
    lb_opt_x['x', i, :n_rooms] = T_R_lb
    ub_opt_x['x', i, :n_rooms] = T_R_ub
    lb_opt_x['x', i, n_rooms:] = T_W_lb+273.15
    ub_opt_x['x', i, n_rooms:] = T_W_ub+273.15

# Set the bounds for inputs
for i in range(N):
    lb_opt_x['u', i, :] = Qdot_lb
    ub_opt_x['u', i, :3] = Qdot_heat_ub
    ub_opt_x['u', i, 3:] = Qdot_cool_ub

# Set bounds for slack variables
lb_opt_x['eps', 0] = eps_lb
lb_opt_x['eps', 1] = eps_lb
ub_opt_x['eps', 0] = eps_ub
ub_opt_x['eps', 1] = eps_ub

# Initialize empty list of constraints
g = []
lb_g = []
ub_g = []

# Enforce initial state with constraint
x_init = SX.sym('x_init', nx)
x0 = opt_x['x', 0]
g.append(x0 - x_init)
lb_g.append(np.zeros((nx,1)))
ub_g.append(np.zeros((nx,1)))

# Initialize Cost
J = 0

# Initialize u_prev
u_prev = u_k_prev 

# Adapt environmental parameters to the horizon
d_N = struct_symSX([entry('d_k', shape=2, repeat=[N])])

for k in range(N):

    # Slack Terms for soft constraints of room temperatures
    slack_lb = slack_penalty*(opt_x['eps', 0, 0]**2 + opt_x['eps', 0, 1]**2 + opt_x['eps', 0, 2]**2)
    slack_ub = slack_penalty*(opt_x['eps', 1, 0]**2 + opt_x['eps', 1, 1]**2 + opt_x['eps', 1, 2]**2)

    # Choose which cost to minimize !!!
    # Update Cost
    J += pseudo_stage_cost_fcn(opt_x['x', k], opt_x['u', k], u_prev) + slack_lb + slack_ub
    # J += economic_stage_cost_fcn(opt_x['u', k], u_prev) + slack_lb + slack_ub

    # Append implicit euler formula as constraint for all horizon steps (system equation)
    gk = opt_x['x', k+1] - opt_x['x', k] - h*system(opt_x['x', k+1], opt_x['u', k], d_N['d_k', k])
    g.append(gk)
    lb_g.append(np.zeros((nx,1)))
    ub_g.append(np.zeros((nx,1)))

    # Append soft constraints with slack variables for room temperatures
    g.append(opt_x['x', k, :n_rooms] - T_R_lb_soft + opt_x['eps', 0]) # T_rooms - 19 + lb_slack >= 0
    lb_g.append(np.zeros((n_rooms,1)))
    ub_g.append(np.ones((n_rooms,1))*np.inf)

    g.append(opt_x['x', k, :n_rooms] - T_R_ub_soft - opt_x['eps', 1]) # T_rooms - 23 - ub_slack <= 0
    lb_g.append(np.ones((n_rooms,1))*-np.inf)
    ub_g.append(np.zeros((n_rooms,1)))

    # Update u_prev
    u_prev = opt_x['u', k]

# Add the terminal constraint to the cost function
# J += terminal_cost_fcn(opt_x['x', N])

g = vertcat(*g)
lb_g = vertcat(*lb_g)
ub_g = vertcat(*ub_g)

opt_problem = {'f':J, 'x':vertcat(opt_x), 'g':g, 'p':vertcat(x_init,d_N,u_k_prev)}
mpc_solver = nlpsol('solver', 'ipopt', opt_problem)

In [ ]:
# MPC Loop: Reference tracking using Kalman Filter estimation

# Generate weather trajectories for MPC
ts_mpc, qs_mpc, tw_mpc, qw_mpc, tt_mpc = generate_weather_trajectory(t_sim+(N_hours/24), dt)

# Initial conditions
x_0 = vertcat(np.ones(n_rooms)*(T_R_0+273.15), np.ones(n_walls)*(T_W_0+273.15))
u_prev_0 = DM([0]*nu)

# Initial condition for observer
x0_observer = vertcat(np.ones(n_rooms)*(20+273.15), np.ones(n_walls)*(18+273.15))

# Process and measurement variances
var_x = 1e-5
var_y = 1e-2

# Initial Guess of Kalman Error Covariance Matrix
P0 = np.eye(nx)

# Initialize lists to store results
res_x_mpc = [x0_observer]
res_u_mpc = [u_prev_0]
cost_mpc = []

# Choose summer or winter
# season = 'summer'
season = 'winter'

for i in range(N_sim):

    # Environmental variables
    if season=='summer':
            d_current = np.concatenate(np.array([[ts_mpc[i:i+N]], [qs_mpc[i:i+N]]]).reshape(2,N).T)
    elif season=='winter':
            d_current = np.concatenate(np.array([[tw_mpc[i:i+N]], [qw_mpc[i:i+N]]]).reshape(2,N).T)
    
    d_k_0 = d_N(d_current)

    p_k = vertcat(x0_observer, d_k_0, u_prev_0)
    
    # Solve optimization problem - Utilize Warm-starting the optimizer 
    if i>0:
        mpc_res = mpc_solver(x0=opt_x_k , p=p_k, lbg=lb_g, ubg=ub_g, lbx = lb_opt_x, ubx = ub_opt_x)
    else:
        mpc_res = mpc_solver(p=p_k, lbg=lb_g, ubg=ub_g, lbx = lb_opt_x, ubx = ub_opt_x)

    # Extract the optimal states and optimal control input
    opt_x_k = opt_x(mpc_res['x'])
    opt_u_k = opt_x_k['u',0]   # we only apply the first input in the horizon

    # Simulate the system to get the measurement for KF, never extract the solution of the optimizer as the next state
    res_integrator = ode_solver(x0=x_0, p=vertcat(opt_u_k, d_k_0['d_k',0]))
    x_next = res_integrator['xf']

    # Get the next state from the Kalman Estimator
    x0_observer, P0, y = kalman_estimator(x_next, x0_observer, opt_u_k, d_k_0['d_k',0], P0, var_x, var_y)

    # Save the cost for plotting
    current_cost = mpc_res['f']

    # Update the initial state
    x_0 = x_next

    # Update the previous input
    u_prev_0 = opt_u_k

    # Store the results
    res_x_mpc.append(x0_observer)
    res_u_mpc.append(opt_u_k)
    cost_mpc.append(current_cost)

# Make an array from the list of arrays:
res_x_mpc = np.concatenate(res_x_mpc,axis=1)
res_u_mpc = np.concatenate(res_u_mpc, axis=1)
cost_mpc = np.concatenate(cost_mpc, axis=1)

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(10,14), sharex=True)
fig.align_ylabels()
# plot the states
ax[0].set_ylabel('Room Temperatures', fontsize=11)
ax[0].plot(res_x_mpc.T[:,:n_rooms]-273.15)
ax[0].axhline(T_R_ub_soft-273.15, color='red', linestyle='--')
ax[0].axhline(T_R_lb_soft-273.15, color='red', linestyle='--')
ax[1].set_ylabel('Wall Temperatures', fontsize=11)
ax[1].plot(res_x_mpc.T[:,n_rooms:]-273.15)
ax[1].axhline(T_W_ub, color='red', linestyle='--')
ax[1].axhline(T_W_lb, color='red', linestyle='--')
ax[2].set_ylabel('Heating Power (kW)', fontsize=11)
ax[2].plot(res_u_mpc.T[:,:3])
ax[2].axhline(Qdot_heat_ub, color='red', linestyle='--')
ax[3].set_ylabel('Cooling Power (kW)', fontsize=11)
ax[3].plot(res_u_mpc.T[:,3:])
ax[3].axhline(Qdot_cool_ub, color='red', linestyle='--')
ax[4].set_ylabel('Closed Loop Cost', fontsize=11)
ax[4].plot(cost_mpc.T)
ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()

plot_states(res_x_mpc, Tenv=ts, q_rad=qs, u_devices=res_u_mpc, plot_constraints=True)


In [ ]:
# MPC Loop: Reference tracking using Kalman Filter estimation

# Generate weather trajectories for MPC
ts_mpc, qs_mpc, tw_mpc, qw_mpc, tt_mpc = generate_weather_trajectory(t_sim+(N_hours/24), dt)

# Initial conditions
x_0 = vertcat(np.ones(n_rooms)*(T_R_0+273.15), np.ones(n_walls)*(T_W_0+273.15))
u_prev_0 = DM([0]*nu)

# Initial condition for observer
x0_observer = vertcat(np.ones(n_rooms)*(20+273.15), np.ones(n_walls)*(18+273.15))

# Process and measurement variances
var_x = 1e-5
var_y = 1e-2

# Initial Guess of Kalman Error Covariance Matrix
P0 = np.eye(nx)

# Initialize lists to store results
res_x_mpc = [x0_observer]
res_u_mpc = [u_prev_0]
cost_mpc = []

# Choose summer or winter
# season = 'summer'
season = 'winter'

for i in range(N_sim):

    # Environmental variables
    if season=='summer':
            d_current = np.concatenate(np.array([[ts_mpc[i:i+N]], [qs_mpc[i:i+N]]]).reshape(2,N).T)
    elif season=='winter':
            d_current = np.concatenate(np.array([[tw_mpc[i:i+N]], [qw_mpc[i:i+N]]]).reshape(2,N).T)
    
    d_k_0 = d_N(d_current)

    p_k = vertcat(x0_observer, d_k_0, u_prev_0)
    
    # Solve optimization problem - Utilize Warm-starting the optimizer 
    if i>0:
        mpc_res = mpc_solver(x0=opt_x_k , p=p_k, lbg=lb_g, ubg=ub_g, lbx = lb_opt_x, ubx = ub_opt_x)
    else:
        mpc_res = mpc_solver(p=p_k, lbg=lb_g, ubg=ub_g, lbx = lb_opt_x, ubx = ub_opt_x)

    # Extract the optimal states and optimal control input
    opt_x_k = opt_x(mpc_res['x'])
    opt_u_k = opt_x_k['u',0]   # we only apply the first input in the horizon

    # Simulate the system to get the measurement for KF, never extract the solution of the optimizer as the next state
    res_integrator = ode_solver(x0=x_0, p=vertcat(opt_u_k, d_k_0['d_k',0]))
    x_next = res_integrator['xf']

    # Get the next state from the Kalman Estimator
    x0_observer, P0, y = kalman_estimator(x_next, x0_observer, opt_u_k, d_k_0['d_k',0], P0, var_x, var_y)

    # Save the cost for plotting
    current_cost = mpc_res['f']

    # Update the initial state
    x_0 = x_next

    # Update the previous input
    u_prev_0 = opt_u_k

    # Store the results
    res_x_mpc.append(x0_observer)
    res_u_mpc.append(opt_u_k)
    cost_mpc.append(current_cost)

# Make an array from the list of arrays:
res_x_mpc = np.concatenate(res_x_mpc,axis=1)
res_u_mpc = np.concatenate(res_u_mpc, axis=1)
cost_mpc = np.concatenate(cost_mpc, axis=1)

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(10,14), sharex=True)
fig.align_ylabels()
# plot the states
ax[0].set_ylabel('Room Temperatures', fontsize=11)
ax[0].plot(res_x_mpc.T[:,:n_rooms]-273.15)
ax[0].axhline(T_R_ub_soft-273.15, color='red', linestyle='--')
ax[0].axhline(T_R_lb_soft-273.15, color='red', linestyle='--')
ax[1].set_ylabel('Wall Temperatures', fontsize=11)
ax[1].plot(res_x_mpc.T[:,n_rooms:]-273.15)
ax[1].axhline(T_W_ub, color='red', linestyle='--')
ax[1].axhline(T_W_lb, color='red', linestyle='--')
ax[2].set_ylabel('Heating Power (kW)', fontsize=11)
ax[2].plot(res_u_mpc.T[:,:3])
ax[2].axhline(Qdot_heat_ub, color='red', linestyle='--')
ax[3].set_ylabel('Cooling Power (kW)', fontsize=11)
ax[3].plot(res_u_mpc.T[:,3:])
ax[3].axhline(Qdot_cool_ub, color='red', linestyle='--')
ax[4].set_ylabel('Closed Loop Cost', fontsize=11)
ax[4].plot(cost_mpc.T)
ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()

plot_states(res_x_mpc, Tenv=tw, q_rad=qw, u_devices=res_u_mpc, plot_constraints=True)


In [ ]:
# MPC Loop: Economic MPC using Kalman Filter estimation

# Generate weather trajectories for MPC
ts_mpc, qs_mpc, tw_mpc, qw_mpc, tt_mpc = generate_weather_trajectory(t_sim+(N_hours/24), dt)

# Initial conditions
x_0 = vertcat(np.ones(n_rooms)*(T_R_0+273.15), np.ones(n_walls)*(T_W_0+273.15))
u_prev_0 = DM([0]*nu)

# Initial condition for observer
x0_observer = vertcat(np.ones(n_rooms)*(20+273.15), np.ones(n_walls)*(18+273.15))

# Process and measurement variances
var_x = 1e-5
var_y = 1e-2

# Initial Guess of Kalman Error Covariance Matrix
P0 = np.eye(nx)

# Initialize lists to store results
res_x_mpc = [x0_observer]
res_u_mpc = [u_prev_0]
cost_mpc = []

# Choose summer or winter
season = 'summer'
# season = 'winter'

for i in range(N_sim):

    # Environmental variables
    if season=='summer':
            d_current = np.concatenate(np.array([[ts_mpc[i:i+N]], [qs_mpc[i:i+N]]]).reshape(2,N).T)
    elif season=='winter':
            d_current = np.concatenate(np.array([[tw_mpc[i:i+N]], [qw_mpc[i:i+N]]]).reshape(2,N).T)
    
    d_k_0 = d_N(d_current)


    p_k = vertcat(x0_observer, d_k_0, u_prev_0)
    
    # Solve optimization problem - Utilize Warm-starting the optimizer 
    if i>0:
        mpc_res = mpc_solver(x0=opt_x_k , p=p_k, lbg=lb_g, ubg=ub_g, lbx = lb_opt_x, ubx = ub_opt_x)
    else:
        mpc_res = mpc_solver(p=p_k, lbg=lb_g, ubg=ub_g, lbx = lb_opt_x, ubx = ub_opt_x)

    # Extract the optimal states and optimal control input
    opt_x_k = opt_x(mpc_res['x'])
    opt_u_k = opt_x_k['u',0]   # we only apply the first input in the horizon

    # Simulate the system to get the measurement for KF, never extract the solution of the optimizer as the next state
    res_integrator = ode_solver(x0=x_0, p=vertcat(opt_u_k, d_k_0['d_k',0]))
    x_next = res_integrator['xf']

    # Get the next state from the Kalman Estimator
    x0_observer, P0, y = kalman_estimator(x_next, x0_observer, opt_u_k, d_k_0['d_k',0], P0, var_x, var_y)

    # Save the cost for plotting
    current_cost = mpc_res['f']

    # Update the initial state
    x_0 = x_next

    # Update the previous input
    u_prev_0 = opt_u_k

    # Store the results
    res_x_mpc.append(x0_observer)
    res_u_mpc.append(opt_u_k)
    cost_mpc.append(current_cost)

# Make an array from the list of arrays:
res_x_mpc = np.concatenate(res_x_mpc,axis=1)
res_u_mpc = np.concatenate(res_u_mpc, axis=1)
cost_mpc = np.concatenate(cost_mpc, axis=1)

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(10,14), sharex=True)
fig.align_ylabels()
# plot the states
ax[0].set_ylabel('Room Temperatures', fontsize=11)
ax[0].plot(res_x_mpc.T[:,:n_rooms]-273.15)
ax[0].axhline(T_R_ub_soft-273.15, color='red', linestyle='--')
ax[0].axhline(T_R_lb_soft-273.15, color='red', linestyle='--')
ax[1].set_ylabel('Wall Temperatures', fontsize=11)
ax[1].plot(res_x_mpc.T[:,n_rooms:]-273.15)
ax[1].axhline(T_W_ub, color='red', linestyle='--')
ax[1].axhline(T_W_lb, color='red', linestyle='--')
ax[2].set_ylabel('Heating Power (kW)', fontsize=11)
ax[2].plot(res_u_mpc.T[:,:3])
ax[2].axhline(Qdot_heat_ub, color='red', linestyle='--')
ax[3].set_ylabel('Cooling Power (kW)', fontsize=11)
ax[3].plot(res_u_mpc.T[:,3:])
ax[3].axhline(Qdot_cool_ub, color='red', linestyle='--')
ax[4].set_ylabel('Closed Loop Cost', fontsize=11)
ax[4].plot(cost_mpc.T)
ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()

plot_states(res_x_mpc, Tenv=ts, q_rad=qs, u_devices=res_u_mpc, plot_constraints=True)

In [ ]:
# MPC Loop: Economic MPC using Kalman Filter estimation

# Generate weather trajectories for MPC
ts_mpc, qs_mpc, tw_mpc, qw_mpc, tt_mpc = generate_weather_trajectory(t_sim+(N_hours/24), dt)

# Initial conditions
x_0 = vertcat(np.ones(n_rooms)*(T_R_0+273.15), np.ones(n_walls)*(T_W_0+273.15))
u_prev_0 = DM([0]*nu)

# Initial condition for observer
x0_observer = vertcat(np.ones(n_rooms)*(20+273.15), np.ones(n_walls)*(18+273.15))

# Process and measurement variances
var_x = 1e-5
var_y = 1e-2

# Initial Guess of Kalman Error Covariance Matrix
P0 = np.eye(nx)

# Initialize lists to store results
res_x_mpc = [x0_observer]
res_u_mpc = [u_prev_0]
cost_mpc = []

# Choose summer or winter
# season = 'summer'
season = 'winter'

for i in range(N_sim):

    # Environmental variables
    if season=='summer':
            d_current = np.concatenate(np.array([[ts_mpc[i:i+N]], [qs_mpc[i:i+N]]]).reshape(2,N).T)
    elif season=='winter':
            d_current = np.concatenate(np.array([[tw_mpc[i:i+N]], [qw_mpc[i:i+N]]]).reshape(2,N).T)
    
    d_k_0 = d_N(d_current)


    p_k = vertcat(x0_observer, d_k_0, u_prev_0)
    
    # Solve optimization problem - Utilize Warm-starting the optimizer 
    if i>0:
        mpc_res = mpc_solver(x0=opt_x_k , p=p_k, lbg=lb_g, ubg=ub_g, lbx = lb_opt_x, ubx = ub_opt_x)
    else:
        mpc_res = mpc_solver(p=p_k, lbg=lb_g, ubg=ub_g, lbx = lb_opt_x, ubx = ub_opt_x)

    # Extract the optimal states and optimal control input
    opt_x_k = opt_x(mpc_res['x'])
    opt_u_k = opt_x_k['u',0]   # we only apply the first input in the horizon

    # Simulate the system to get the measurement for KF, never extract the solution of the optimizer as the next state
    res_integrator = ode_solver(x0=x_0, p=vertcat(opt_u_k, d_k_0['d_k',0]))
    x_next = res_integrator['xf']

    # Get the next state from the Kalman Estimator
    x0_observer, P0, y = kalman_estimator(x_next, x0_observer, opt_u_k, d_k_0['d_k',0], P0, var_x, var_y)

    # Save the cost for plotting
    current_cost = mpc_res['f']

    # Update the initial state
    x_0 = x_next

    # Update the previous input
    u_prev_0 = opt_u_k

    # Store the results
    res_x_mpc.append(x0_observer)
    res_u_mpc.append(opt_u_k)
    cost_mpc.append(current_cost)

# Make an array from the list of arrays:
res_x_mpc = np.concatenate(res_x_mpc,axis=1)
res_u_mpc = np.concatenate(res_u_mpc, axis=1)
cost_mpc = np.concatenate(cost_mpc, axis=1)

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(10,14), sharex=True)
fig.align_ylabels()
# plot the states
ax[0].set_ylabel('Room Temperatures', fontsize=11)
ax[0].plot(res_x_mpc.T[:,:n_rooms]-273.15)
ax[0].axhline(T_R_ub_soft-273.15, color='red', linestyle='--')
ax[0].axhline(T_R_lb_soft-273.15, color='red', linestyle='--')
ax[1].set_ylabel('Wall Temperatures', fontsize=11)
ax[1].plot(res_x_mpc.T[:,n_rooms:]-273.15)
ax[1].axhline(T_W_ub, color='red', linestyle='--')
ax[1].axhline(T_W_lb, color='red', linestyle='--')
ax[2].set_ylabel('Heating Power (kW)', fontsize=11)
ax[2].plot(res_u_mpc.T[:,:3])
ax[2].axhline(Qdot_heat_ub, color='red', linestyle='--')
ax[3].set_ylabel('Cooling Power (kW)', fontsize=11)
ax[3].plot(res_u_mpc.T[:,3:])
ax[3].axhline(Qdot_cool_ub, color='red', linestyle='--')
ax[4].set_ylabel('Closed Loop Cost', fontsize=11)
ax[4].plot(cost_mpc.T)
ax[0].grid()
ax[1].grid()
ax[2].grid()
ax[3].grid()
ax[4].grid()

plot_states(res_x_mpc, Tenv=tw, q_rad=qw, u_devices=res_u_mpc, plot_constraints=True)